In [10]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [11]:
files = glob("../outputs/*/*/prometheus-13b-v1.0/ultrachat-test.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        score = score.replace(":", "")
        if score:
            return int(score)
        else:
            return -1
    else:
        return -1

for file in files:
    df = pd.read_json(file, lines=True)
    if "judge" in df.columns:
        df["judge_score"] = df.judge.apply(filter_score)
    else:
        df["chosen_judge_score"] = df.judge_chosen.apply(filter_score)
        df["rejected_judge_score"] = df.judge_rejected.apply(filter_score)
    dfs.append(df)

df = pd.concat(dfs)
df.shape

ValueError: No objects to concatenate

In [12]:
df.groupby("generator").agg({
    "judge_score": ["mean", "median", "std", "count"],
})

judge_score         \
                                                          mean median   
generator                                                               
heegyu/0430-DFO-1.1b-CCFT@epoch-1                          NaN    NaN   
heegyu/0430-DFO-1.1b-DPO@epoch-1                         1.645    1.0   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...       1.620    1.0   

                                                                    
                                                         std count  
generator                                                           
heegyu/0430-DFO-1.1b-CCFT@epoch-1                        NaN     0  
heegyu/0430-DFO-1.1b-DPO@epoch-1                    1.263555  1000  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.249464  1000

In [8]:
df.groupby("generator").agg({
    "chosen_judge_score": ["mean", "median", "std", "count"],
    "rejected_judge_score": ["mean", "median", "std", "count"],
})

chosen_judge_score         \
                                                                 mean median   
generator                                                                      
heegyu/0430-DFO-1.1b-CCFT@epoch-1                               1.848    1.0   
heegyu/0430-DFO-1.1b-DPO@epoch-1                                  NaN    NaN   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...                NaN    NaN   

                                                                  \
                                                       std count   
generator                                                          
heegyu/0430-DFO-1.1b-CCFT@epoch-1                   1.4481  1000   
heegyu/0430-DFO-1.1b-DPO@epoch-1                       NaN     0   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...     NaN     0   

                                                   rejected_judge_score  \
                                                                   mean   
generator                                                                 
heegyu/0430-DFO-1.1b-CCFT@epoch-1                                 1.799   
heegyu/0430-DFO-1.1b-DPO@epoch-1                                    NaN   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...                  NaN   

                                                                           
                                                   median       std count  
generator                                                                  
heegyu/0430-DFO-1.1b-CCFT@epoch-1                     1.0  1.413009  1000  
heegyu/0430-DFO-1.1b-DPO@epoch-1                      NaN       NaN     0  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...    NaN       NaN     0

In [44]:
df[df.score > 0].groupby("generator").agg({
    "score": "mean",
    "output_len": ["mean", "median", "std", "count"],
})

score   output_len  \
                                                        mean         mean   
generator                                                                   
heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.784826  1156.945274   
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.520548  1336.073474   
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1.436567  1274.685323   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1  1.829390  1035.331258   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2  1.737888   981.081988   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3  1.603727  1107.804969   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.558739  1086.747851   
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1.618634  1237.853416   
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1.513682  1294.924129   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.869240  1041.265255   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.901863  1040.172671   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.776119  1133.180348   
text_davinci_003                                    1.778882  1059.672050   

                                                                              
                                                    median         std count  
generator                                                                     
heegyu/TinyLlama-1.1b-feedback-all@epoch-1           990.5  901.824627   804  
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1195.0  918.657444   803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1136.0  909.712958   804  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1   893.0  800.633616   803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2   797.0  793.368868   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3   918.0  879.540126   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   920.0  853.469184   349  
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1107.0  866.385656   805  
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1169.5  889.523366   804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   892.0  825.778962   803  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   897.0  822.370682   805  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   987.0  878.581902   804  
text_davinci_003                                     956.0  783.421212   805

# PairRM

In [3]:
files = glob("../outputs/*/*/pairrm/*.json")
dfs = []

for file in files:
    df = pd.read_json(file, lines=True)
    df["dataset"] = os.path.basename(file).split(".")[0]
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(9780, 6)

In [7]:
df.groupby(["dataset", "generator"]).agg({
    "judge": ["mean", "median", "std", "count"],
})

judge  \
                                                                   mean   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.149573   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.218166   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.611811   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -3.949996   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.393866   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.331274   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.657126   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -5.949652   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.698814   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.780337   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.670845   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.336723   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.290145   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.174094   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.410455   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.185481   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.247956   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.745288   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.755498   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.836002   
           text_davinci_003                                   -3.869928   

                                                                         \
                                                                 median   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.185547   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.335938   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.738281   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -4.121094   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.478516   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.300781   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.960938   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -6.359375   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.826172   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.875000   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.857422   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.525391   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.414062   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.203125   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.480469   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.343750   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.367188   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.880859   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.910156   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.935547   
           text_davinci_003                                   -4.000000   

                                                                               
                                                                    std count  
dataset    generator                                                           
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.350452   276  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.435997   361  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1